## **Minimax and Alpha-beta pruning**

In [26]:
import pygame
from pygame.locals import *
import sys
import numpy as np
import math
import random

# Constants
ROW = 6
COL = 7
SQUARESIZE = 100
RADIUS = int(SQUARESIZE / 2 - 5)
WIDTH = COL * SQUARESIZE
HEIGHT = (ROW + 1) * SQUARESIZE
SIZE = (WIDTH, HEIGHT)
WHITE = "DarkGrey"  # Background color
Player1 = "coral1"
player2 = "cornsilk2"
empty = "Burlywood3"
AI = 0
PLAYER = 1
PLAYER_PIECE = 2
AI_PIECE = 1
EMPTY_PLACE = 0
EASY_LEVEL = 1
MEDIUM_LEVEL = 2
HARD_LEVEL = 4

# Functions
def board_is_full(board):
    for c in range(COL):
        for r in range(ROW):
            if board[r][c] == 0:
                return False
    return True
#Create an empty game board matrix 
def create_board_matrix(ROW, COL):
    return np.zeros((ROW, COL), dtype=np.int8)
#if column is not full  and a piece can be placed in
def is_valid_location(board, col):
    return board[ROW - 1][col] == EMPTY_PLACE
#check winner move
def is_Winning_situation(board, pieceColor):
    # horizontal
    for c in range(COL - 3):
        for r in range(ROW):
            if board[r][c] == pieceColor and board[r][c + 1] == pieceColor and board[r][c + 2] == pieceColor and \
                    board[r][c + 3] == pieceColor:
                return True
    # vertical
    for c in range(COL):
        for r in range(ROW - 3):
            if board[r][c] == pieceColor and board[r + 1][c] == pieceColor and board[r + 2][c] == pieceColor and \
                    board[r + 3][c] == pieceColor:
                return True
    # diagonal positive
    for c in range(COL - 3):
        for r in range(ROW - 3):
            if board[r][c] == pieceColor and board[r + 1][c + 1] == pieceColor and board[r + 2][c + 2] == pieceColor and \
                    board[r + 3][c + 3] == pieceColor:
                return True
    # diagonal negative
    for c in range(COL - 3):
        for r in range(3, ROW):
            if board[r][c] == pieceColor and board[r - 1][c + 1] == pieceColor and board[r - 2][c + 2] == pieceColor and \
                    board[r - 3][c + 3] == pieceColor:
                return True
           
#Create a Pygame display window based on size of game board
def create_Display(ROW, COL):
    return pygame.display.set_mode((COL * SQUARESIZE, (ROW + 1) * SQUARESIZE))

#Draws the game board on the Pygame surface.
def draw_board(boardSurface, board):
    no_rows, no_cols = board.shape
    boardSurface.fill(WHITE)
    for i in range(no_rows):
        for j in range(no_cols):
            pygame.draw.rect(boardSurface, empty, (j * SQUARESIZE, i * SQUARESIZE + SQUARESIZE, SQUARESIZE, SQUARESIZE))
            if board[i, j] == 0:
                pygame.draw.circle(boardSurface, WHITE, (int(j * SQUARESIZE + SQUARESIZE / 2), int(i * SQUARESIZE + SQUARESIZE + SQUARESIZE / 2)), RADIUS)
            elif board[i, j] == 1:
                pygame.draw.circle(boardSurface, Player1, (int(j * SQUARESIZE + SQUARESIZE / 2), int(i * SQUARESIZE + SQUARESIZE + SQUARESIZE / 2)), RADIUS)
            elif board[i, j] == 2:
                pygame.draw.circle(boardSurface, player2, (int(j * SQUARESIZE + SQUARESIZE / 2), int(i * SQUARESIZE + SQUARESIZE + SQUARESIZE / 2)), RADIUS)
            else:
                print("Error: unresolved board value")
    pygame.display.update()
#Inserts a piece into the game board at the specified column
def insert_piece(board, col, pieceColor, boardBackground):
    inserted_flag = 0
    for i in range(ROW - 1, -1, -1):
        if board[i, col] == 0:
            board[i, col] = pieceColor
            inserted_flag = 1
            if i > 0:
                board[i - 1, col] = 0
            draw_board(boardBackground, board)
            pygame.time.wait(50)
            break
    return board, inserted_flag
#
def Human_player(board, boardBackground):
    insertionFlag = 0
    winner_flag = 0
    while insertionFlag == 0:
        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
                sys.exit()
            if event.type == MOUSEBUTTONDOWN:
                pygame.draw.rect(boardBackground, WHITE, (0, 0, SQUARESIZE * COL, SQUARESIZE))
                clickPosition = event.pos[0]
                insertionCol = int(math.floor(clickPosition / SQUARESIZE))
                board, insertionFlag = insert_piece(board, insertionCol, PLAYER_PIECE, boardBackground)
                if is_Winning_situation(board, PLAYER_PIECE):
                    winner_flag = 1
                return board, winner_flag
            if event.type == pygame.MOUSEMOTION:
                pygame.draw.rect(boardBackground, WHITE, (0, 0, SQUARESIZE * COL, SQUARESIZE))
                col_pos = event.pos[0]
                pygame.draw.circle(boardBackground, empty, (col_pos, int(SQUARESIZE / 2)), RADIUS)
                pygame.display.update()
    return board, winner_flag

#Adds a piece to the maatrix without updating the display
def addPiecetoMatrixWithoutDrawing(board, col, pieceColor):
    for i in range(ROW - 1, -1, -1):
        if board[i, col] == 0:
            board[i, col] = pieceColor
            break
    return board
#Returns a list of incomplete columns.
def getInCompleteColumns(board):
    inCompleteColumns = []
    for c in range(COL):
        if board[0][c] == 0:
            inCompleteColumns.append(c)
    return inCompleteColumns
#Assigns scores to different piece configurations in a line.
def operate_on_line(line, game_level):
    score = 0
    if game_level == EASY_LEVEL:
        if line.count(AI_PIECE) == 4:
            score = 70
        elif line.count(AI_PIECE) == 3 and line.count(EMPTY_PLACE) == 1:
            score = 30
        elif line.count(AI_PIECE) == 2 and line.count(EMPTY_PLACE) == 2:
            score = 20
        elif line.count(PLAYER_PIECE) == 4 and line.count(EMPTY_PLACE) == 0:
            score = -80
        elif line.count(PLAYER_PIECE) == 3 and line.count(EMPTY_PLACE) == 1:
            score = -50
        elif line.count(PLAYER_PIECE) == 2 and line.count(EMPTY_PLACE) == 2:
            score = -15
    # Add similar cases for other game levels
    return score
#Calculates the overall score of a board state.
def get_score(board, game_level):
    totalScore = 0
    for r in range(ROW):
        for c in range(COL - 3):
            totalScore = totalScore + operate_on_line([board[r][c], board[r][c + 1], board[r][c + 2], board[r][c + 3]], game_level)
    for c in range(COL):
        for r in range(ROW - 3):
            totalScore = totalScore + operate_on_line([board[r][c], board[r + 1][c], board[r + 2][c], board[r + 3][c]], game_level)
    for c in range(COL - 3):
        for r in range(ROW - 3):
            totalScore = totalScore + operate_on_line([board[r][c], board[r + 1][c + 1], board[r + 2][c + 2], board[r + 3][c + 3]], game_level)
    for c in range(c - 3):
        for r in range(3, ROW):
            totalScore = totalScore + operate_on_line([board[r][c], board[r - 1][c + 1], board[r - 2][c + 2], board[r - 3][c + 3]], game_level)
    if game_level != EASY_LEVEL:
        colCenter = COL // 2
        for r in range(3, ROW):
            if board[r][colCenter] == AI_PIECE:
                totalScore = totalScore + 5
    return totalScore
#Implements the AI's decision-making using the minimax and alpha-beta pruning algorithm
def minimax(board, level, maximizingPlayer, alpha, beta, game_level):
    inCompleteColumns = getInCompleteColumns(board)
    if level == 0:
        return (None, get_score(board, game_level))
    elif is_Winning_situation(board, PLAYER_PIECE):
        return (None, -math.inf)
    elif is_Winning_situation(board, AI_PIECE):
        return (None, math.inf)
    elif len(getInCompleteColumns(board)) == 0:
        return (None, 0)
    elif maximizingPlayer:
        bestScore = -math.inf
        chosenColumn = random.choice(inCompleteColumns)
        for c in inCompleteColumns:
            temp = board.copy()
            addPiecetoMatrixWithoutDrawing(temp, c, AI_PIECE)
            newScore = minimax(temp, level - 1, False, alpha, beta, game_level)[1]
            if newScore > bestScore:
                bestScore = newScore
                chosenColumn = c
            alpha = max(alpha, bestScore)
            if alpha >= beta:
                break
        return chosenColumn, bestScore
    else:
        bestScore = math.inf
        chosenColumn = random.choice(inCompleteColumns)
        for c in inCompleteColumns:
            temp = board.copy()
            addPiecetoMatrixWithoutDrawing(temp, c, PLAYER_PIECE)
            newScore = minimax(temp, level - 1, True, alpha, beta, game_level)[1]
            if newScore < bestScore:
                bestScore = newScore
                chosenColumn = c
            beta = min(beta, bestScore)
            if alpha >= beta:
                break
        return chosenColumn, bestScore
#Handles the AI's turn by using minimax and alpha-beta pruning calculations
def AI_takes_turn(board, boardBackGround, Level):
    col = minimax(board, Level, True, -math.inf, math.inf, Level)[0]
    winner_flag = 0
    insertionFlag = 0
    while insertionFlag == 0:
        board, insertionFlag = insert_piece(board, col, AI_PIECE, boardBackGround)
        if is_Winning_situation(board, AI_PIECE):
            winner_flag = 1
    return board, winner_flag

def main():
    board = create_board_matrix(ROW, COL)
    display = create_Display(ROW, COL)
    pygame.init()

    while True:
        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
                sys.exit()

        draw_board(display, board)

        if not is_Winning_situation(board, PLAYER_PIECE) and not is_Winning_situation(board, AI_PIECE) and not board_is_full(board):
            board, player_wins = Human_player(board, display)
            if player_wins:
                print("Player wins!")
                break

            board, ai_wins = AI_takes_turn(board, display, Level=3)
            if ai_wins:
                print("AI wins!")
                break

    pygame.quit()

if __name__ == "__main__":
    main()


AI wins!


# **Hill Climbing and minimax algorithm**


In [25]:
import pygame
from pygame.locals import *
import sys
import numpy as np
import math
import random

# Constants
ROW = 6
COL = 7
SQUARESIZE = 100
RADIUS = int(SQUARESIZE / 2 - 5)
WIDTH = COL * SQUARESIZE
HEIGHT = (ROW + 1) * SQUARESIZE
SIZE = (WIDTH, HEIGHT)
WHITE = "DimGrey"  # Background color
Player1 = "LightPink1"
player2 = "LightBlue2"
empty = "lavender"
AI = 0
PLAYER = 1
PLAYER_PIECE = 2
AI_PIECE = 1
EMPTY_PLACE = 0
EASY_LEVEL = 1
MEDIUM_LEVEL = 2
HARD_LEVEL = 4

# Functions
def board_is_full(board):
    for c in range(COL):
        for r in range(ROW):
            if board[r][c] == 0:
                return False
    return True

def create_board_matrix(ROW, COL):
    return np.zeros((ROW, COL), dtype=np.int8)

def is_valid_location(board, col):
    return board[ROW - 1][col] == EMPTY_PLACE

def is_Winning_situation(board, pieceColor):
    # horizontal
    for c in range(COL - 3):
        for r in range(ROW):
            if board[r][c] == pieceColor and board[r][c + 1] == pieceColor and board[r][c + 2] == pieceColor and \
                    board[r][c + 3] == pieceColor:
                return True
    # vertical
    for c in range(COL):
        for r in range(ROW - 3):
            if board[r][c] == pieceColor and board[r + 1][c] == pieceColor and board[r + 2][c] == pieceColor and \
                    board[r + 3][c] == pieceColor:
                return True
    # diagonal positive
    for c in range(COL - 3):
        for r in range(ROW - 3):
            if board[r][c] == pieceColor and board[r + 1][c + 1] == pieceColor and board[r + 2][c + 2] == pieceColor and \
                    board[r + 3][c + 3] == pieceColor:
                return True
    # diagonal negative
    for c in range(COL - 3):
        for r in range(3, ROW):
            if board[r][c] == pieceColor and board[r - 1][c + 1] == pieceColor and board[r - 2][c + 2] == pieceColor and \
                    board[r - 3][c + 3] == pieceColor:
                return True

def create_Display(ROW, COL):
    return pygame.display.set_mode((COL * SQUARESIZE, (ROW + 1) * SQUARESIZE))

def draw_board(boardSurface, board):
    no_rows, no_cols = board.shape
    boardSurface.fill(WHITE)
    for i in range(no_rows):
        for j in range(no_cols):
            pygame.draw.rect(boardSurface, empty, (j * SQUARESIZE, i * SQUARESIZE + SQUARESIZE, SQUARESIZE, SQUARESIZE))
            if board[i, j] == 0:
                pygame.draw.circle(boardSurface, WHITE, (int(j * SQUARESIZE + SQUARESIZE / 2), int(i * SQUARESIZE + SQUARESIZE + SQUARESIZE / 2)), RADIUS)
            elif board[i, j] == 1:
                pygame.draw.circle(boardSurface, Player1, (int(j * SQUARESIZE + SQUARESIZE / 2), int(i * SQUARESIZE + SQUARESIZE + SQUARESIZE / 2)), RADIUS)
            elif board[i, j] == 2:
                pygame.draw.circle(boardSurface, player2, (int(j * SQUARESIZE + SQUARESIZE / 2), int(i * SQUARESIZE + SQUARESIZE + SQUARESIZE / 2)), RADIUS)
            else:
                print("Error: unresolved board value")
    pygame.display.update()

def insert_piece(board, col, pieceColor, boardBackground):
    inserted_flag = 0
    for i in range(ROW - 1, -1, -1):
        if board[i, col] == 0:
            board[i, col] = pieceColor
            inserted_flag = 1
            if i > 0:
                board[i - 1, col] = 0
            draw_board(boardBackground, board)
            pygame.time.wait(50)
            break
    return board, inserted_flag

def Human_player(board, boardBackground):
    insertionFlag = 0
    winner_flag = 0
    while insertionFlag == 0:
        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
                sys.exit()
            if event.type == MOUSEBUTTONDOWN:
                pygame.draw.rect(boardBackground, WHITE, (0, 0, SQUARESIZE * COL, SQUARESIZE))
                clickPosition = event.pos[0]
                insertionCol = int(math.floor(clickPosition / SQUARESIZE))
                board, insertionFlag = insert_piece(board, insertionCol, PLAYER_PIECE, boardBackground)
                if is_Winning_situation(board, PLAYER_PIECE):
                    winner_flag = 1
                return board, winner_flag
            if event.type == pygame.MOUSEMOTION:
                pygame.draw.rect(boardBackground, WHITE, (0, 0, SQUARESIZE * COL, SQUARESIZE))
                col_pos = event.pos[0]
                pygame.draw.circle(boardBackground, empty, (col_pos, int(SQUARESIZE / 2)), RADIUS)
                pygame.display.update()
    return board, winner_flag

def addPiecetoMatrixWithoutDrawing(board, col, pieceColor):
    for i in range(ROW - 1, -1, -1):
        if board[i, col] == 0:
            board[i, col] = pieceColor
            break
    return board

def getInCompleteColumns(board):
    inCompleteColumns = []
    for c in range(COL):
        if board[0][c] == 0:
            inCompleteColumns.append(c)
    return inCompleteColumns

def operate_on_line(line, game_level):
    score = 0
    if game_level == EASY_LEVEL:
        if line.count(AI_PIECE) == 4:
            score = 70
        elif line.count(AI_PIECE) == 3 and line.count(EMPTY_PLACE) == 1:
            score = 30
        elif line.count(AI_PIECE) == 2 and line.count(EMPTY_PLACE) == 2:
            score = 20
        elif line.count(PLAYER_PIECE) == 4 and line.count(EMPTY_PLACE) == 0:
            score = -80
        elif line.count(PLAYER_PIECE) == 3 and line.count(EMPTY_PLACE) == 1:
            score = -50
        elif line.count(PLAYER_PIECE) == 2 and line.count(EMPTY_PLACE) == 2:
            score = -15
    # Add similar cases for other game levels
    return score

def get_score(board, game_level):
    totalScore = 0
    for r in range(ROW):
        for c in range(COL - 3):
            totalScore = totalScore + operate_on_line([board[r][c], board[r][c + 1], board[r][c + 2], board[r][c + 3]], game_level)
    for c in range(COL):
        for r in range(ROW - 3):
            totalScore = totalScore + operate_on_line([board[r][c], board[r + 1][c], board[r + 2][c], board[r + 3][c]], game_level)
    for c in range(COL - 3):
        for r in range(ROW - 3):
            totalScore = totalScore + operate_on_line([board[r][c], board[r + 1][c + 1], board[r + 2][c + 2], board[r + 3][c + 3]], game_level)
    for c in range(c - 3):
        for r in range(3, ROW):
            totalScore = totalScore + operate_on_line([board[r][c], board[r - 1][c + 1], board[r - 2][c + 2], board[r - 3][c + 3]], game_level)
    if game_level != EASY_LEVEL:
        colCenter = COL // 2
        for r in range(3, ROW):
            if board[r][colCenter] == AI_PIECE:
                totalScore = totalScore + 5
    return totalScore

# New functions for Hill Climbing
def evaluate_move(board, col, pieceColor, game_level):
    temp_board = board.copy()
    addPiecetoMatrixWithoutDrawing(temp_board, col, pieceColor)
    return get_score(temp_board, game_level)

def hill_climbing(board, level, game_level):
    inCompleteColumns = getInCompleteColumns(board)
    best_score = -math.inf
    best_move = random.choice(inCompleteColumns)

    for col in inCompleteColumns:
        score = evaluate_move(board, col, AI_PIECE, game_level)
        if score > best_score:
            best_score = score
            best_move = col

    return best_move

def AI_takes_turn_hill_climbing(board, boardBackGround, level):
    col = hill_climbing(board, level, level)
    winner_flag = 0
    insertionFlag = 0
    while insertionFlag == 0:
        board, insertionFlag = insert_piece(board, col, AI_PIECE, boardBackGround)
        if is_Winning_situation(board, AI_PIECE):
            winner_flag = 1
    return board, winner_flag

def main():
    board = create_board_matrix(ROW, COL)
    display = create_Display(ROW, COL)
    pygame.init()

    while True:
        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
                sys.exit()

        draw_board(display, board)

        if not is_Winning_situation(board, PLAYER_PIECE) and not is_Winning_situation(board, AI_PIECE) and not board_is_full(board):
            board, player_wins = Human_player(board, display)
            if player_wins:
                print("Player wins!")
                break

            board, ai_wins = AI_takes_turn_hill_climbing(board, display, level=3)
            if ai_wins:
                print("AI wins!")
                break

    pygame.quit()

if __name__ == "__main__":
    main()


Player wins!
